In [10]:
import nltk
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

In [11]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head(20)

,id,raw_address,POI/street
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika
1,1,"aye, jati sampurna",/
2,2,setu siung 119 rt 5 1 13880 cipayung,/siung
3,3,"toko dita, kertosono",toko dita/
4,4,jl. orde baru,/jl. orde baru
5,5,"raya samb gede, 299 toko bb kids",toko bb kids/raya samb gede
6,6,"kem mel raya, no 4 bojong rawalumbu rt 1 36 ra...",/kem mel raya
7,7,tela keuramat kuta alam,/tela
8,8,gg. i wates magersari,/gg. i
9,9,bunga ncole ix 2,/bunga ncole ix


In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 3 columns):
id             300000 non-null int64
raw_address    300000 non-null object
POI/street     300000 non-null object
dtypes: int64(1), object(2)
memory usage: 6.9+ MB


In [15]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
id             50000 non-null int64
raw_address    50000 non-null object
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [12]:
#define tokenizer
def tokenizer(col):
    tokens = []
    for address in col:
        token = nltk.word_tokenize(address)
        tokens.append(token)
    return tokens

raw_address = list(train['raw_address'])
POI = []
street = []

for stuff in train['POI/street']:
    stuff_list = list(stuff.split('/'))
    POI.append(stuff_list[0])
    street.append(stuff_list[1])
    
#raw_address_t = tokenizer(raw_address)
#POI_t = tokenizer(POI)
#street_t = tokenizer(street)

In [13]:
train['POI'] = POI
train['street'] = street

In [9]:
train

,id,raw_address,POI/street,POI,street
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika,,jl kapuk timur delta sili iii lippo cika
1,1,"aye, jati sampurna",/,,
2,2,setu siung 119 rt 5 1 13880 cipayung,/siung,,siung
3,3,"toko dita, kertosono",toko dita/,toko dita,
4,4,jl. orde baru,/jl. orde baru,,jl. orde baru
...,...,...,...,...,...
299995,299995,jend ahmad yani 331 kertasari ciamis,/jend ahmad yani,,jend ahmad yani
299996,299996,"raya cila kko, cilandak timur kel.",/raya cila kko,,raya cila kko
299997,299997,tanjung gusta jl. yaya 2 no 17,/,,
299998,299998,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri/,taman asri,


In [14]:
# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
fulltext = train['raw_address']
vectorizer.fit(fulltext)
# summarize
#print(vectorizer.vocabulary_)
print(vectorizer.idf_)
# encode document
vector_raw = vectorizer.transform(fulltext)
vector_POI = vectorizer.transform(POI)
vector_street = vectorizer.transform(street)
# summarize encoded vector
print(vector_raw.shape)
print(vector_POI.shape)
print(vector_street.shape)
print(vector_raw)
print(vector_POI)
print(vector_street)

[10.14580518 11.12663444  7.40698332 ... 12.91839391 12.91839391
 12.91839391]
(300000, 81117)
(300000, 81117)
(300000, 81117)
  (0, 74750)	0.16180615691387076
  (0, 68311)	0.4182858166323804
  (0, 60862)	0.27102741656170026
  (0, 45061)	0.35617193115768225
  (0, 38481)	0.27388440098044975
  (0, 36727)	0.15957651778976625
  (0, 33901)	0.20053903431301487
  (0, 24277)	0.31469154370991304
  (0, 21593)	0.24343002437787079
  (0, 21554)	0.30372954137682967
  (0, 21341)	0.4121058078385727
  (0, 442)	0.19358829533915445
  (1, 65233)	0.5141720636248635
  (1, 35948)	0.3775193544687558
  (1, 14237)	0.7701339013374514
  (2, 69012)	0.5631596034403585
  (2, 67451)	0.3463506071567673
  (2, 63901)	0.14191221672150028
  (2, 22032)	0.31527996464436986
  (2, 941)	0.5235585631277244
  (2, 588)	0.41138596912109726
  (3, 75292)	0.2920168080044063
  (3, 40563)	0.6392728631386438
  (3, 25197)	0.7113764054967232
  (4, 54410)	0.8913958240011443
  :	:
  (299997, 80162)	0.5341902850080407
  (299997, 73114)	0.338

In [15]:
model = Sequential()

In [17]:
X_train, X_test, y_train, y_test = train_test_split(vector_raw, vector_POI, test_size = 0.20, random_state = 69)

In [18]:
history = model.fit(X_train, y_train, batch_size = 128, epochs = 6, verbose = 1, validation_split = 0.2)

ValueError: Please provide as model inputs either a single array or a list of arrays. You passed: x=  (0, 65836)	0.45659440645595734
  (0, 62908)	0.5757834032761974
  (0, 33516)	0.4469571653125465
  (0, 21346)	0.3683525764613141
  (0, 13003)	0.3529032336135456
  (1, 62169)	0.2869974158875874
  (1, 57018)	0.7181261043855556
  (1, 20647)	0.6339774297826961
  (2, 76906)	0.35803458636547913
  (2, 68793)	0.4475343963186378
  (2, 65069)	0.36389749945935457
  (2, 64870)	0.4162730255805638
  (2, 33065)	0.4119040590398416
  (2, 10534)	0.3348433737047928
  (2, 1951)	0.289887113151896
  (3, 75292)	0.35636538507909554
  (3, 61467)	0.9343466767305527
  (4, 80615)	0.5842739831761078
  (4, 57110)	0.6357445205043555
  (4, 25910)	0.31724244874898505
  (4, 15328)	0.3921862388508295
  (5, 77316)	0.4730753934723785
  (5, 66646)	0.2557850002825782
  (5, 59479)	0.3939608356690084
  (5, 52980)	0.1602997230692474
  :	:
  (239996, 75292)	0.34799422946243325
  (239996, 52980)	0.2101068817763208
  (239996, 48539